In [1]:
# --- 0. Importações ---
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [2]:
# --- 1. Carregamento e Preparação dos Dados (Conforme solicitado) ---
# Carrega o dataset "g-credit" (ID 31) do OpenML usando fetch_openml
# O 'as_frame=True' já retorna X e y como DataFrames/Series do pandas.
dataset = fetch_openml(data_id=31, as_frame=True, parser='auto')
X = dataset.data
y = dataset.target

# A classe alvo "good" é a classe 1 (majoritária) e "bad" é a 0 (minoritária)
# Para scikit-learn, é bom garantir que as classes sejam numéricas
y = y.cat.codes # 'good' -> 1, 'bad' -> 0

In [3]:
# --- 2. Definição do Pré-processamento ---
# Identifica colunas numéricas e categóricas
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['category', 'object']).columns

# Cria o transformador de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

In [4]:
# --- 3. Criação do Pipeline com SMOTE e XGBoost ---
# O pipeline irá:
# 1. Pré-processar os dados
# 2. Aplicar SMOTE para balancear as classes (APENAS nos dados de treino)
# 3. Treinar o classificador XGBoost
model_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

In [7]:
# --- 4. Otimização de Hiperparâmetros com GridSearchCV e StratifiedKFold ---
# Define os parâmetros para testar.
param_grid = {
    'smote__k_neighbors': [3, 5, 7],
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.05, 0.1],
    # O scale_pos_weight é um excelente parâmetro para desbalanceamento no XGBoost
    'classifier__scale_pos_weight': [1, y.value_counts()[0] / y.value_counts()[1]]
}

# Define a estratégia de validação cruzada: 10-fold estratificado (NOVA ALTERAÇÃO)
# Shuffle=True garante que os dados sejam embaralhados antes de dividir em folds
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

# Configura o GridSearchCV para buscar os melhores parâmetros
# Otimizando para a métrica AUC-ROC
grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=cv, # Usando a estratégia de 10 folds
    n_jobs=-1,
    verbose=1
)

# Executa a busca pelos melhores hiperparâmetros
grid_search.fit(X, y)



Fitting 20 folds for each of 72 candidates, totalling 1440 fits


c:\projects\statistics-main-folder\german-credit\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
1 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\projects\statistics-main-folder\german-credit\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\projects\statistics-main-folder\german-credit\.venv\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\projects\statistic

GridSearchCV(cv=StratifiedKFold(n_splits=20, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['duration', 'credit_amount', 'installment_commitment',
       'residence_since', 'age', 'existing_credits', 'num_dependents'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEnco...
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [0.05, 0.1],
                         'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [100, 200],
                         'classifier__scale_pos_weight': [1,
                                                          np.float64(0.42857142857142855)],
                         'smote__k_neighbors': [3, 5, 7]},
             scoring='roc_auc', verbose=1)

In [9]:
# --- 5. Avaliação do Melhor Modelo ---
print("\n" + "="*50)
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
print(f"Melhor pontuação AUC-ROC na validação cruzada (10-fold): {grid_search.best_score_:.4f}\n")

# Usa o melhor modelo encontrado para fazer previsões no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)
y_pred_proba = best_model.predict_proba(X)[:, 1]

# Exibe os resultados finais
print("--- Resultados no Conjunto de Teste (Dados nunca vistos) ---")
print(f"AUC-ROC: {roc_auc_score(y, y_pred_proba):.4f}")
print("\nMatriz de Confusão:")
# [[Verdadeiro Negativo, Falso Positivo],
#  [Falso Negativo,    Verdadeiro Positivo]]
print(confusion_matrix(y, y_pred))
print("\nRelatório de Classificação:")
# '0' é 'bad credit' (minoritária), '1' é 'good credit' (majoritária)
print(classification_report(y, y_pred, target_names=['bad credit (0)', 'good credit (1)']))
print("="*50)


Melhores parâmetros encontrados: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 200, 'classifier__scale_pos_weight': 1, 'smote__k_neighbors': 3}
Melhor pontuação AUC-ROC na validação cruzada (10-fold): 0.7981

--- Resultados no Conjunto de Teste (Dados nunca vistos) ---
AUC-ROC: 0.9603

Matriz de Confusão:
[[237  63]
 [ 32 668]]

Relatório de Classificação:
                 precision    recall  f1-score   support

 bad credit (0)       0.88      0.79      0.83       300
good credit (1)       0.91      0.95      0.93       700

       accuracy                           0.91      1000
      macro avg       0.90      0.87      0.88      1000
   weighted avg       0.90      0.91      0.90      1000

